In [14]:
import torch
import numpy as np


In [15]:
# Load NN

model = IsingNet()
model.load_state_dict(torch.load("/home/vv/IsingModelPractice/NN/rand_train_set.pt"))
model.eval()

NameError: name 'IsingNet' is not defined

In [16]:
xy = np.loadtxt("/home/vv/IsingModelPractice/Data/Energy w All_4x4_flat.csv", delimiter=",", dtype=np.float32, skiprows=1)
y = [i for i in xy[:,0]]
x = [[i] for i in xy[:,1:17]]

In [17]:
posbl_abs_E = [0,1,4,8,12,16,20,24,32] #28 isn't possible
NN_errs_per_state = {}

for i in posbl_abs_E:
    NN_errs_per_state[i]=[]




[[] for i in range(8)]
print(NN_err_per_state)


# this chooses the closest class of predicted value
# i.e. regression to classification
def closest_abs_E(y_pred):
    acc = np.abs(y_pred)
    best = 0
    for i in posbl_abs_E:
        err = np.abs(i-np.abs(y_pred))
        if err <= acc:
            acc = err
            best = i
    return best


for i in x:
    y_pred = model(i)
    y_pred_class = closest_abs_E(y_pred)
    NN_errs_per_state[y_pred_class].append(np.abs(y-y_pred_class))

[[], [], [], [], [], [], [], []]


NameError: name 'model' is not defined